# Set Up

In [1]:
# Data Processing Packagesx
import yfinance as yf
import pandas as pd
import numpy as np
import numpy as np
# -----------------------

# ML Packages
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import accuracy_score
# -----------------------

# Quantum Computing Packages
from pennylane.optimize import NesterovMomentumOptimizer
import pennylane as qml
from pennylane.kernels import square_kernel_matrix, kernel_matrix
import pennylane as qml
# -----------------------

# Feature Engineering

In [2]:
# Import Data

ticker = "TSLA"
start = "2020-01-01"
end = "2025-01-01"
interval = "1d"
stock_data = yf.download(ticker, start, end, interval, auto_adjust = False)

[*********************100%***********************]  1 of 1 completed


In [3]:
stock_data

Price        Adj Close       Close Dividends        High         Low  \
Ticker            TSLA        TSLA      TSLA        TSLA        TSLA   
Date                                                                   
2020-01-02   28.684000   28.684000       0.0   28.713333   28.114000   
2020-01-03   29.534000   29.534000       0.0   30.266666   29.128000   
2020-01-06   30.102667   30.102667       0.0   30.104000   29.333332   
2020-01-07   31.270666   31.270666       0.0   31.441999   30.224001   
2020-01-08   32.809334   32.809334       0.0   33.232666   31.215334   
...                ...         ...       ...         ...         ...   
2024-12-24  462.279999  462.279999       0.0  462.779999  435.140015   
2024-12-26  454.130005  454.130005       0.0  465.329987  451.019989   
2024-12-27  431.660004  431.660004       0.0  450.000000  426.500000   
2024-12-30  417.410004  417.410004       0.0  427.000000  415.750000   
2024-12-31  403.839996  403.839996       0.0  427.929993  402.540009   

Price             Open Stock Splits     Volume  
Ticker            TSLA         TSLA       TSLA  
Date                                            
2020-01-02   28.299999          0.0  142981500  
2020-01-03   29.366667          0.0  266677500  
2020-01-06   29.364668          0.0  151995000  
2020-01-07   30.760000          0.0  268231500  
2020-01-08   31.580000          0.0  467164500  
...                ...          ...        ...  
2024-12-24  435.899994          0.0   59551800  
2024-12-26  465.160004          0.0   76366400  
2024-12-27  449.519989          0.0   82666800  
2024-12-30  419.399994          0.0   64941000  
2024-12-31  423.790009          0.0   76825100  

[1258 rows x 8 columns]

In [4]:
# Feature Selection

stock_features = stock_data[["Open", "High", "Low", "Adj Close", "Volume"]].copy()
stock_features["Log Returns"] = np.log(stock_features["Adj Close"]/stock_features["Adj Close"].shift(1))
stock_features["Price Change"] = stock_features["Adj Close"] - stock_features["Adj Close"].shift(1)
target = (stock_features["Adj Close"].shift(-1) > stock_features["Adj Close"]).astype(int)
data = stock_features.copy()
data['target'] = target

# Clean

data = data.dropna()
features_clean = data.drop(columns=['target'])
labels_clean = data['target']


# Scale
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_clean)

label_array = labels_clean.to_numpy()





/var/folders/y_/n4ck7tvs3f521rw37q7654400000gn/T/ipykernel_25802/838504062.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  features_clean = data.drop(columns=['target'])


# Preliminary Test

In [5]:
x_train, x_test, y_train, y_test = train_test_split(features_scaled, label_array, test_size = .2)

In [6]:
# Create Qubits
n_qubits = x_train.shape[1] # Select qubits = to the number of columns
dev = qml.device("default.qubit", wires=n_qubits) # We are simulating a quantum system with pennylanes default

In [7]:
# Angle Encoding Circut
@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits)) # template that applies rotation aroundy axis by default!
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) # calculates the hermitian adjoint (inverse)
    return qml.probs(wires=range(n_qubits))



# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


We use angle encoding to map classical data into quantum states by creating SU(2) rotations from the input features and applying them to an initial quantum state.

In [8]:
%time
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)


# Metrics
y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")


CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 μs
Wall time: 2.86 μs
ROC AUC: 0.5693435268844379
PR AUC: 0.6182518865431719
Precision: 0.5277777777777778
Recall: 1.0
F1 Score: 0.6909090909090909
Accuracy: 0.5277777777777778


In [9]:
%time
# Optimization: Introduce Entaglement in Feature Maps (CNOT Entaglement)

@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))

    # CNOT ENTAGLEMENT
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) 
    return qml.probs(wires=range(n_qubits))


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)


y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.15 μs
ROC AUC: 0.5342768686421937
PR AUC: 0.5623431602891837
Precision: 0.5277777777777778
Recall: 1.0
F1 Score: 0.6909090909090909
Accuracy: 0.5277777777777778


In [10]:
%time
# Optimization: Introduce Entaglement in Feature Maps (XX Entaglement)

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising Entaglment
    for i in range(n_qubits - 1):
        qml.IsingXX(np.pi / 2, wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("PR AUC:", pr_auc)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.15 μs
Accuracy: 0.5277777777777778
PR AUC: 0.5623431602891837
Precision: 0.5277777777777778
Recall: 1.0
F1 Score: 0.6909090909090909
Accuracy: 0.5277777777777778


In [11]:
%time
# Optimization With Layers

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising entanglement between neighbors
    for _ in range(n_layers):  
        for i in range(n_qubits - 1):
            qml.IsingXX(np.pi / 2, wires=[i, i + 1])
        for i in range(n_qubits):
            qml.RY(np.pi / 4, wires=i)  
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("PR AUC:", pr_auc)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.91 μs
Accuracy: 0.5277777777777778
PR AUC: 0.5623431602891837
Precision: 0.5277777777777778
Recall: 1.0
F1 Score: 0.6909090909090909
Accuracy: 0.5277777777777778


# Further Feature Engineering


## Momentum Indicators

In [17]:
# Reinit Data (Want to Drop NA after adding new features)
data = stock_features.copy()
data['target'] = target

In [18]:
# Realative Strenght Index
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data["RSI"] = compute_rsi(data["Adj Close"])


In [19]:
# Momentum 
data["Momentum"] = data["Adj Close"] - data["Adj Close"].shift(10)

In [20]:
data = data.dropna()
features_clean = data.drop(columns=['target'])
labels_clean = data['target']


# Scale
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_clean)

label_array = labels_clean.to_numpy()

/var/folders/y_/n4ck7tvs3f521rw37q7654400000gn/T/ipykernel_25802/3821203942.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  features_clean = data.drop(columns=['target'])


In [21]:
x_train, x_test, y_train, y_test = train_test_split(features_scaled, label_array, test_size = .2)

n_qubits = x_train.shape[1] # Select qubits = to the number of columns
dev = qml.device("default.qubit", wires=n_qubits)
# Angle Encoding Circut
@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits)) # template that applies rotation aroundy axis by default!
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) # calculates the hermitian adjoint (inverse)
    return qml.probs(wires=range(n_qubits))



# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]


In [22]:
%time

x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)



# Metrics
from sklearn.metrics import accuracy_score

y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")





CPU times: user 3 μs, sys: 1 μs, total: 4 μs
Wall time: 5.72 μs
ROC AUC: 0.569250645994832
PR AUC: 0.5744516634110582
Precision: 0.5180722891566265
Recall: 1.0
F1 Score: 0.6825396825396826
Accuracy: 0.5180722891566265


### Entagnlement Tests

In [23]:
%time
# Optimization: Introduce Entaglement in Feature Maps (CNOT Entaglement)

@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))

    # CNOT ENTAGLEMENT
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) 
    return qml.probs(wires=range(n_qubits))


# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)


y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")


CPU times: user 4 μs, sys: 2 μs, total: 6 μs
Wall time: 11.2 μs
ROC AUC: 0.487532299741602
PR AUC: 0.5008802175333189
Precision: 0.5180722891566265
Recall: 1.0
F1 Score: 0.6825396825396826
Accuracy: 0.5180722891566265


In [24]:
%time
# Optimization: Introduce Entaglement in Feature Maps (XX Entaglement)

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising Entaglment
    for i in range(n_qubits - 1):
        qml.IsingXX(np.pi / 2, wires=[i, i + 1])
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("PR AUC:", pr_auc)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 1.91 μs
Accuracy: 0.5180722891566265
PR AUC: 0.5008802175333189
Precision: 0.5180722891566265
Recall: 1.0
F1 Score: 0.6825396825396826
Accuracy: 0.5180722891566265


In [25]:
%time
# Optimization With Layers

@qml.qnode(dev)
def circuit(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    
    # Ising entanglement between neighbors
    for _ in range(n_layers):  
        for i in range(n_qubits - 1):
            qml.IsingXX(np.pi / 2, wires=[i, i + 1])
        for i in range(n_qubits):
            qml.RY(np.pi / 4, wires=i)  
    
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Kernal Matices
x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("PR AUC:", pr_auc)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

CPU times: user 1 μs, sys: 1e+03 ns, total: 2 μs
Wall time: 5.01 μs
Accuracy: 0.5180722891566265
PR AUC: 0.5008802175333189
Precision: 0.5180722891566265
Recall: 1.0
F1 Score: 0.6825396825396826
Accuracy: 0.5180722891566265


### Remove Linear Indicator

In [26]:
# Run Without Momentum
data = stock_features.copy()
data['target'] = target
data["RSI"] = compute_rsi(data["Adj Close"])

data = data.dropna()
features_clean = data.drop(columns=['target'])
labels_clean = data['target']


# Scale
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_clean)

label_array = labels_clean.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(features_scaled, label_array, test_size = .2)

n_qubits = x_train.shape[1] # Select qubits = to the number of columns
dev = qml.device("default.qubit", wires=n_qubits)


%time

x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)



# Metrics
from sklearn.metrics import accuracy_score

y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

/var/folders/y_/n4ck7tvs3f521rw37q7654400000gn/T/ipykernel_25802/2683126632.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  features_clean = data.drop(columns=['target'])


CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 4.05 μs
ROC AUC: 0.49230769230769234
PR AUC: 0.5346264234611503
Precision: 0.5220883534136547
Recall: 1.0
F1 Score: 0.6860158311345647
Accuracy: 0.5220883534136547


In [28]:
# Without RSI

data = stock_features.copy()
data['target'] = target
data["Momentum"] = data["Adj Close"] - data["Adj Close"].shift(10)

data = data.dropna()
features_clean = data.drop(columns=['target'])
labels_clean = data['target']


# Scale
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_clean)

label_array = labels_clean.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(features_scaled, label_array, test_size = .2)

n_qubits = x_train.shape[1] # Select qubits = to the number of columns
dev = qml.device("default.qubit", wires=n_qubits)


%time

x_train_kernel = square_kernel_matrix(x_train, kernel=quantum_kernel)
x_test_kernel = kernel_matrix(x_test, x_train, kernel=quantum_kernel)

# Train the SVM
clf = SVC(kernel='precomputed')
clf.fit(x_train_kernel, y_train)

y_pred = clf.predict(x_test_kernel)



# Metrics
from sklearn.metrics import accuracy_score

y_scores = clf.decision_function(x_test_kernel)
roc_auc = roc_auc_score(y_test, y_scores)
pr_auc = average_precision_score(y_test, y_scores)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
accuracy = accuracy_score(y_test, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

/var/folders/y_/n4ck7tvs3f521rw37q7654400000gn/T/ipykernel_25802/3118848659.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  features_clean = data.drop(columns=['target'])


CPU times: user 1 μs, sys: 1e+03 ns, total: 2 μs
Wall time: 5.25 μs
ROC AUC: 0.5448434799308624
PR AUC: 0.5668982366038341
Precision: 0.508
Recall: 1.0
F1 Score: 0.6737400530503979
Accuracy: 0.508
